1. docx 안에 이미지를 GPT가 읽지 못하므로 표로 바꾸어서 넣어준다.
2. 이 때 표를 table로 그리면 역시 알아내지 못한다. 때문에 GPT가 알아들을 수 있도록 markdown 양식으로 바꿔준다.
3. 미리 스포를 하자면 3-5를 진행해도 GPT는 제대로 답변을 주지 않는다. 3-6에 가서야 나온다.

In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./tax_with_markdown.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [2]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [ ]:
import os
import time

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

pinecone_api_key = os.environ.get("PINECONE_API_KEY")
index_name='tax-markdown-index'

pc=Pinecone(api_key=pinecone_api_key)

#database=PineconeVectorStore.from_documents(document_list,embedding,index_name=index_name)

# 먼저 database 객체 생성
database = PineconeVectorStore(index_name=index_name, embedding=embedding)

# documents를 나누어 업로드 (예: 100개씩 배치 처리)
batch_size = 100
for i in range(0, len(document_list), batch_size):
    batch = document_list[i:i + batch_size]
    database.add_documents(batch)

In [4]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

In [ ]:
retriever = database.as_retriever(search_kwargs={'k': 4})
retriever.invoke(query)

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [7]:
#prompt
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [8]:
ai_message = qa_chain.invoke({"query": query})

In [ ]:
ai_message
